In [3]:
DIR=%pwd
%cd $DIR/radsa/

/home/bgx/.mega2/Desarrollo/github/radsa


In [4]:
%%writefile dataproc-cuda.py

# Antena Data Processor 
import numpy as np
import sys
import argparse
from bitarray import bitarray
from libradsa import *

#CUDA packages
import pycuda.driver as cuda
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import pycuda.driver as drv
from pycuda import cumath
import numpy as np
import skcuda.linalg as linalg
import skcuda.misc as misc
import ctypes


def main(args):
    qbits=args.qbits
    
    datastream = bitarray()
    if args.infile:
        with args.infile as f:
            datastream.fromfile(f)
    
    ## Read data
    chl0,chl1,chl2,chl3=demux(datastream)
    
    if qbits == 1:
        decode1bit()
    elif qbits == 2:
        CH0,CH1,CH2,CH3=decode2bit(chl0,chl1,chl2,chl3)
    elif qbits == 4:
        CH0,CH1,CH2,CH3=decode4bit(chl0,chl1,chl2,chl3)
    elif qbits == 8:
        CH0,CH1,CH2,CH3=decode8bit(chl0,chl1,chl2,chl3)
    elif qbits == 16:
        CH0,CH1,CH2,CH3=decode16bit(chl0,chl1,chl2,chl3)
   



    a = np.array([CH0.view(np.complex64).astype(np.complex128),
         CH1.view(np.complex64).astype(np.complex128),
         CH2.view(np.complex64).astype(np.complex128),
         CH3.view(np.complex64).astype(np.complex128)])
    
    N = np.int32(a.shape[1])
    
    linalg.init()
    x_gpu = gpuarray.to_gpu(a)
    y_gpu = linalg.conj(x_gpu)
    z_gpu = linalg.multiply(x_gpu, y_gpu)
    x_gpu.gpudata.free()
    y_gpu.gpudata.free()
    RMS_gpu=cumath.sqrt(misc.sum(z_gpu,axis=1,keepdims=True)/N)
    RMS=RMS_gpu.get()
    RMS_gpu.gpudata.free()
   
    print args.infile.name,(RMS.real).flatten()
    
if __name__ == "__main__":

    import time
    start_time = time.time()
    parser = argparse.ArgumentParser(description='Simulate antena data format.')
    parser.add_argument('-q', dest='qbits', type=int, choices=[2,4,8,16],
                        help="Bits quantization", required=True)
    parser.add_argument('-i', '--infile', dest='infile',type=argparse.FileType('r'),
                        help='Read from IN_FILE the simulated data.', required=True)
    args = parser.parse_args()
    
    main(args)
    print("--- %s seconds ---" % (time.time() - start_time))

Overwriting dataproc-cuda.py


In [3]:
%run dataproc.py {'-q 2 -i simdata.bin'}

[-1  0  0  1 -1 -2 -1 -1]
[ 0 -2  0 -1  0 -1  0 -1]
[-1  0 -1 -1 -1 -2  0  0]
[ 0 -1 -1 -1 -1 -2  0  0]


In [19]:
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
import numpy
from pycuda.curandom import rand as curand

a = (numpy.random.randn(400)
        +1j*numpy.random.randn(400)).astype(numpy.complex64)

a_gpu = gpuarray.to_gpu(a)

from pycuda.elementwise import ElementwiseKernel
complex_mul = ElementwiseKernel(
        "pycuda::complex<float> *x, pycuda::complex<float> *y, pycuda::complex<float> *z",
        "z[i] = x[i] * y[i]",
        "complex_mul",
        preamble="#include <pycuda-complex.hpp>",)

c_gpu = gpuarray.empty_like(a_gpu)
complex_mul(a_gpu,a_gpu, c_gpu)

import numpy.linalg as la
error = la.norm(c_gpu.get() - (a*a))
print error
assert error < 1e-5


1.04577e-06


In [19]:
import pycuda.driver as drv
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
import numpy as np
import skcuda.linalg as linalg
import ctypes

# ctypes.CDLL('libgomp.so.1', mode=ctypes.RTLD_GLOBAL)
# ctypes.cdll.LoadLibrary('libcusolver.so')

linalg.init()
x = np.array([[1+1j, 2-2j, 3+3j, 4-4j], [5+5j, 6-6j, 7+7j, 8-8j]], np.complex64)
x_gpu = gpuarray.to_gpu(x)
y_gpu = linalg.conj(x_gpu)

print x
y=np.conjugate(x)
print y
print x_gpu
print y_gpu

z_gpu = linalg.multiply(x_gpu, y_gpu)
print z_gpu
z=x*y
print z
np.all(x == np.conj(y_gpu.get()))

[[ 1.+1.j  2.-2.j  3.+3.j  4.-4.j]
 [ 5.+5.j  6.-6.j  7.+7.j  8.-8.j]]
[[ 1.-1.j  2.+2.j  3.-3.j  4.+4.j]
 [ 5.-5.j  6.+6.j  7.-7.j  8.+8.j]]
[[ 1.+1.j  2.-2.j  3.+3.j  4.-4.j]
 [ 5.+5.j  6.-6.j  7.+7.j  8.-8.j]]
[[ 1.-1.j  2.+2.j  3.-3.j  4.+4.j]
 [ 5.-5.j  6.+6.j  7.-7.j  8.+8.j]]
[[   2.+0.j    8.+0.j   18.+0.j   32.+0.j]
 [  50.+0.j   72.+0.j   98.+0.j  128.+0.j]]
[[   2.+0.j    8.+0.j   18.+0.j   32.+0.j]
 [  50.+0.j   72.+0.j   98.+0.j  128.+0.j]]


True